<a href="https://colab.research.google.com/github/Zheng-Ao/Colab-Notebooks/blob/main/P0_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 全局设置

In [ ]:
'''常用库导入'''
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
raw_data_path = "drive/MyDrive/P0/T10K.csv"
# 测试谷歌云端硬盘是否成功加载:
# pd.read_csv(raw_data_path)

from sklearn import metrics

import torch
print(torch.__version__)                                                        # 用于确定PyG的安装
from torch import nn
from torch.utils.data import Dataset, Subset, ConcatDataset, DataLoader

# 检查GPU:
# !nvidia-smi
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
%%capture
'''特殊库安装(魔法函数要防止Cell的最开始)'''
# 我不想输出安装信息，虽然%%capture不是用来干这个的，但它能达到我想要的效果
!pip install transformers
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-geometric

# UTILS.py

## Txt Embedding

In [ ]:
'''构建txt_vecs，作为Dataset的第二个数据来源'''
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")      
nlp_model = DistilBertModel.from_pretrained("distilbert-base-uncased").to(device)
'''加载BERT这种大模型很耗时，因此在整个流程中应当让上面两行代码只执行一次。'''

In [ ]:
df = pd.read_csv(raw_data_path)

for i in tqdm(range(10000)):
    ttl = df["patent_title"].values[i]
    inputs = tokenizer(ttl, return_tensors="pt").to(device)
    outputs = nlp_model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    cls_vec = last_hidden_states[:,0,:].clone().detach()
    if i == 0:
        txt_vecs = cls_vec
    else:
        txt_vecs = torch.cat([txt_vecs, cls_vec],dim=0)

print(txt_vecs.shape)
# should be [num_samples, embedding_dim] = [10000, 768]

In [ ]:
Mat = txt_vecs.cpu().numpy()

from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler

# txt_pca = PCA(n_components=0.9, svd_solver = 'full')
txt_pca = PCA(n_components = 5)

In [ ]:
# 查看PCA的效果:
txt_pca.fit(Mat)
var = txt_pca.explained_variance_ratio_
len(var), var.sum()

In [ ]:
Txt_Embedding = txt_pca.fit_transform(Mat)
print(Txt_Embedding.shape)
# should be [10000, 5] here

## utils

In [ ]:
from datetime import datetime
# target transform, 0:neg, 1:pos
def LabelCorePa(ref, isd):
    now = datetime.strptime("2022-01-01", "%Y-%m-%d").year
    years = now - datetime.strptime(isd, "%Y-%m-%d").year
    score = ref/years       
    label = int((score>0.5))
    return label

# Dataset
class PatDataset(Dataset):
    def __init__(self, raw_data_path, txt_vecs, transform = None, target_transform = LabelCorePa):
        self.raw_data = pd.read_csv(raw_data_path)
        self.txt_vecs = txt_vecs
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, idx):
        # Y          
        ref = self.raw_data.at[idx, "patent_num_cited_by_us_patents"]
        isd = self.raw_data.at[idx, "patent_date"]
        if self.target_transform:
            label = self.target_transform(ref, isd)
            label = torch.tensor(label,dtype=torch.long)

        # X
        # INDEXs
        num_claims = self.raw_data.at[idx, "patent_num_claims"]
        num_b_cits = self.raw_data.at[idx, "patent_num_us_patent_citations"]
        inventors = self.raw_data.at[idx, "inventors"]
        num_inventors = len(eval(inventors))
        assignees = self.raw_data.at[idx, "assignees"]
        if assignees == "[{'assignee_sequence': None, 'assignee_key_id': None}]":
            num_assignees = 0
        else:
            num_assignees = len(eval(assignees))
        IPCs = self.raw_data.at[idx, "IPCs"]
        num_ipcs = len(eval(IPCs))
        indexs = torch.tensor([num_claims, num_b_cits, num_inventors, num_assignees, num_ipcs], dtype=torch.float32).to(device)
        # TXT
        txt_vec = torch.tensor(self.txt_vecs[idx], dtype = torch.float32)
        txt_vec = txt_vec.to(device)

        patent = torch.cat([indexs, txt_vec])
        
        return patent, label

# Models
class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNet, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            # nn.BatchNorm1d(input_size),
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        logits = self.conv2(x, edge_index)                   # (num_nodes, 2)
        return logits                  # (num_nodes, 2)



# Train
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):

        # batch: 第几个batch；X: 包含batch_size个feature vec.
        X = X.to(device)
        y = y.to(device)

        output = model(X).to(device)
        loss = loss_fn(output, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:                                                     # train_size/batch_size = 100, 每20个batch输出一次结果，共输出5次。
            loss, current = loss.item(), batch * len(X)                         
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Test
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X = X.to(device)
            y = y.to(device)
            output = model(X)
            # test_loss += loss_fn(output, y).item()
            # correct += (output.argmax(1) == y).type(torch.float).sum().item()
            
            pred = output.argmax(1).cpu()
            y = y.cpu()
            if batch == 0:
                Pred = pred
                Y = y
            else:
                Pred = torch.cat((Pred, pred), dim = 0)
                Y = torch.cat((Y, y), dim=0)

    C_Mat = metrics.confusion_matrix(Y, Pred)
    accuracy = metrics.accuracy_score(Y,Pred)
    f1 = metrics.f1_score(Y, Pred)
    recall = metrics.recall_score(Y, Pred)
    precision = metrics.precision_score(Y, Pred)
    print(C_Mat)
    print(f"acc:{accuracy:.4f}, f1:{f1:.4f}, recall:{recall:.4f}, prec:{precision:.4f}")

    # test_loss /= num_batches
    # correct /= size
    # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# config.py

In [ ]:
# HyperParams----Config.py
hidden_size = 16
output_size = 2

learning_rate = 1e-3
weight_decay = 5e-4
batch_size = 64
epochs = 100
num_epochs_to_print = epochs/10                                                 # 每隔10次输出一次Metrics

num_train = 8000
num_test = 2000

# Main

## Net

In [ ]:
# PIPLINE
# DATA TO FIT A MODEL
dataset = PatDataset(raw_data_path=raw_data_path, txt_vecs = Txt_Embedding)
training_indices = [i for i in range(num_train)]
test_indices = [i for i in range(num_train,num_train+num_test)]
training_data_all = Subset(dataset, training_indices)
test_data = Subset(dataset, test_indices)

# Make pos:neg in training set 1:1
pos_indices = []
neg_indices = []
for i in range(num_train):
    if training_data_all[i][1] == 1:
        pos_indices.append(i)
    else:
        neg_indices.append(i)
num_pos = len(pos_indices)
num_neg = len(neg_indices)
print(f"NEG:POS = {num_neg}:{num_pos} = {num_neg/num_pos:.2f}")
neg_indices_sample = np.random.choice(neg_indices, num_pos, replace = False)	 
training_data_pos = Subset(training_data_all, pos_indices)
training_data_neg = Subset(training_data_all, neg_indices_sample)
training_data = ConcatDataset([training_data_pos, training_data_neg])
print("Training sample number:",len(training_data))

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [ ]:
# FIT A MODEL
# PyTorch的逻辑是先初始化（喂超参），再进行函数计算（喂输入）
input_size = len(dataset[0][0])
model = SimpleNet(input_size, hidden_size, output_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

for t in range(epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer)
    if t==0 or (t+1)%num_epochs_to_print == 0:
        print(f"Epoch {t+1}\n-------------------------------")
        test_loop(train_dataloader, model, loss_fn)                                 # performance on training data
        test_loop(test_dataloader, model, loss_fn)                                  # performance on test data
print("Done!")

## PyG GNN Net

In [ ]:
# edge_index, edge_attr
df = pd.read_csv(raw_data_path)

adj_ls = []
edges = []

for i in tqdm(range(10000)):
    id = df.values[i][0]
    b_cits = eval(df["cited_patents"].values[i])
    for b_cit in b_cits:
        # 是否在专利数据库中
        if b_cit["cited_patent_date"]:
            # 是否在本数据集中
            if b_cit["cited_patent_number"] in df["patent_number"].values:
                # print(i)
                cit_id = df[df["patent_number"].values == b_cit["cited_patent_number"]].values[0][0]
                adj_ls.append([id, cit_id])
                date = df["patent_date"].values[id]
                cit_date = df["patent_date"].values[cit_id]
                date1 = datetime.strptime(date, "%Y-%m-%d")
                date2 = datetime.strptime(cit_date, "%Y-%m-%d")
                dist = date1-date2
                dist = 365/dist.days
                edges.append(dist)

In [ ]:
# df[df["patent_number"].values == '3930276'].values[0][0]
# "3930276" in df["patent_number"].values
# edges
ls = [i for i in range(10)]

ls_1 = np.random.choice(ls,5, replace = False)
ls_1

In [ ]:
# data.x, data.y
for i in range(10000):
    if i == 0:
        x = dataset[i][0].unsqueeze(0)
        y = dataset[i][1].unsqueeze(0)
    else:
        x = torch.cat([x, dataset[i][0].unsqueeze(0)])
        y = torch.cat([y, dataset[i][1].unsqueeze(0)])

# 

In [ ]:
x.shape, x.dtype, y.shape, y.dtype

In [ ]:
print(len(pos_indices),len(neg_indices_sample))
print(neg_indices_sample)
# len = 10
# indices = [1,2,4,5]
 
# ls1 = [False]*10
# t = torch.tensor(ls1)
# t[indices] = True
# t

# for i in range(len(pos_indices)):
#     if pos_indices[i] == neg_indices_sample[i]:
#         print(i)

ls = [False]*10000

# ls[pos_indices] = True

ls_t = torch.tensor(ls, dtype = bool)
# ls_t[pos_indices] = True
ls_t[neg_indices_sample] = True
ls_t.sum()


In [ ]:
from torch_geometric.data import Data

# num_nodes, num_node_features
x = torch.tensor(x, dtype=torch.float32)

# num_nodes, 1
y = torch.tensor(y, dtype=torch.long)

# 2, num_edges
edge_index = torch.tensor(adj_ls, dtype=torch.long)

# num_edges, num_edge_features
edge_attr = torch.tensor(edges, dtype=torch.float32)

# train-test split
train_ls = [False]*num_train + [False]*num_test
train_mask = torch.tensor(train_ls, dtype=bool)
train_mask[pos_indices] = True
train_mask[neg_indices_sample] = True
test_ls = [False]*num_train + [True]*num_test
test_mask = torch.tensor(test_ls, dtype=bool)


data = Data(x=x, y=y, edge_index = edge_index.t().contiguous(), edge_attr=edge_attr, train_mask = train_mask, test_mask=test_mask)
data = data.to(device)

In [ ]:
print(f'Number of nodes: {data.num_nodes}') # 节点数量
print(f'Number of edges: {data.num_edges}') # 边数量
print(f'Number of node features: {data.num_node_features}') # 节点属性的维度
print(f'Number of node features: {data.num_features}') # 同样是节点属性的维度
print(f'Number of edge features: {data.num_edge_features}') # 边属性的维度
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}') # 平均节点度
print(f'if edge indices are ordered and do not contain duplicate entries.: {data.is_coalesced()}') # 是否边是有序的同时不含有重复的边
print(f'Number of training nodes: {data.train_mask.sum()}') # 用作训练集的节点
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}') # 用作训练集的节点的数量
print(f'Contains isolated nodes: {data.has_isolated_nodes()}') # 此图是否包含孤立的节点
print(f'Contains self-loops: {data.has_self_loops()}')  # 此图是否包含自环的边
print(f'Is undirected: {data.is_undirected()}')  # 此图是否是无向图

In [ ]:
input_size = len(dataset[0][0])

model = GNN(input_size, hidden_size, output_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = nn.CrossEntropyLoss()

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')
Pred = pred[data.test_mask].cpu()
Y = data.y[data.test_mask].cpu()
C_Mat = metrics.confusion_matrix(Y, Pred)
accuracy = metrics.accuracy_score(Y,Pred)
f1 = metrics.f1_score(Y, Pred)
recall = metrics.recall_score(Y, Pred)
precision = metrics.precision_score(Y, Pred)
print(C_Mat)
print(f"acc:{accuracy:.4f}, f1:{f1:.4f}, recall:{recall:.4f}, prec:{precision:.4f}")


# Save&Load Model

In [ ]:
# 保存模型权重至当前文件夹
torch.save(model.state_dict(), 'model_weights.pth')                             

In [ ]:
model = SimpleNet(input_size, hidden_size, output_size)                         # 需要是同一个模型
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

'''
be sure to call model.eval() method before inferencing to set the dropout and batch normalization layers to evaluation mode. 
Failing to do this will yield inconsistent inference results.
'''

In [ ]:
# 直接保存/加载整个模型
torch.save(model, 'model.pth')

In [ ]:
model = torch.load('model.pth')

# Discoveries
1. SimpleNet的训练情况似乎说明**模型并不能从数据中学到什么东西（特征可能没有提供什么信息）**。